In [1]:
import pandas as pd
import numpy as np
from data_partition import DistributedDataSet
from keras.utils.np_utils import to_categorical
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


2022-03-01 00:42:33.129890: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-03-01 00:42:33.129964: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-K234JIGI): /proc/driver/nvidia/version does not exist


In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

In [4]:
train = pd.read_csv("dataset/train.csv")
test = pd.read_csv("dataset/test.csv")

In [5]:
len(train)

42000

In [6]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
dd = DistributedDataSet(train, 0, 225, 8)
df_di, test = dd.get_distributed_dataset(0.1, 'label')

In [8]:
df_di[0][0]

,y,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
372,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
444,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
450,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
849,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1022,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37585,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
37625,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
37638,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
37652,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
def data_preprocessing(train, test, y_label):
    X_train = train.drop(y_label, axis=1)
    y_train = train[y_label]
    
    X_test = test.drop(y_label, axis=1)
    y_test = test[y_label]
    
    #Normalization
    X_train = X_train/255.0
    X_test = X_test/255.0
    
    X_train = X_train.values.reshape(-1,28,28,1)
    X_test = X_test.values.reshape(-1,28,28,1)
    
    y_train = to_categorical(y_train, num_classes = 10)
    y_test = to_categorical(y_test, num_classes=10)
    
    return X_train, y_train, X_test, y_test

In [10]:
xtr, ytr, xte, yte = data_preprocessing(df_di[0][0], test, 'y')

In [11]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

2022-03-01 00:42:43.978204: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

/home/hitesh/blockFL-sim/virtualenvironment/blockFL/lib/python3.8/site-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [13]:
# learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
#                                             patience=3, 
#                                             verbose=1, 
#                                             factor=0.5, 
#                                             min_lr=0.00001)

In [14]:
epochs = 20 # Turn epochs to 30 to get 0.9967 accuracy
batch_size = 86

In [15]:
history = model.fit(xtr, ytr, batch_size = batch_size, epochs = epochs, validation_data = (xte, yte), verbose = 2)

Epoch 1/20
3/3 - 3s - loss: 2.3207 - accuracy: 0.1091 - val_loss: 2.2573 - val_accuracy: 0.1079 - 3s/epoch - 1s/step
Epoch 2/20
3/3 - 2s - loss: 2.2098 - accuracy: 0.1864 - val_loss: 1.8265 - val_accuracy: 0.6024 - 2s/epoch - 638ms/step
Epoch 3/20
3/3 - 2s - loss: 1.8325 - accuracy: 0.4273 - val_loss: 1.5093 - val_accuracy: 0.6688 - 2s/epoch - 635ms/step
Epoch 4/20
3/3 - 2s - loss: 1.2795 - accuracy: 0.5955 - val_loss: 2.7050 - val_accuracy: 0.2726 - 2s/epoch - 634ms/step
Epoch 5/20
3/3 - 2s - loss: 1.7401 - accuracy: 0.5409 - val_loss: 0.6796 - val_accuracy: 0.8290 - 2s/epoch - 644ms/step
Epoch 6/20
3/3 - 2s - loss: 0.6723 - accuracy: 0.8045 - val_loss: 0.5951 - val_accuracy: 0.8331 - 2s/epoch - 564ms/step
Epoch 7/20
3/3 - 2s - loss: 0.5106 - accuracy: 0.8182 - val_loss: 0.5004 - val_accuracy: 0.8586 - 2s/epoch - 547ms/step
Epoch 8/20
3/3 - 2s - loss: 0.4276 - accuracy: 0.8500 - val_loss: 0.5920 - val_accuracy: 0.8012 - 2s/epoch - 532ms/step
Epoch 9/20
3/3 - 2s - loss: 0.5002 - accura

In [16]:
weights = model.get_weights()

In [20]:
model.set_weights(weights)

In [21]:
model.fit(xtr, ytr, batch_size = batch_size, epochs = 2, validation_data = (xte, yte), verbose = 2)

Epoch 1/2
3/3 - 2s - loss: 0.0963 - accuracy: 0.9545 - val_loss: 0.3913 - val_accuracy: 0.9048 - 2s/epoch - 604ms/step
Epoch 2/2
3/3 - 2s - loss: 0.0508 - accuracy: 0.9864 - val_loss: 0.3819 - val_accuracy: 0.9052 - 2s/epoch - 547ms/step


In [22]:
weights2 = model.get_weights()

In [25]:
avg_weights = np.average([weights, weights2], axis=0)

In [26]:
model.set_weights(avg_weights)

In [27]:
model.fit(xtr, ytr, batch_size = batch_size, epochs = 2, validation_data = (xte, yte), verbose = 2)

Epoch 1/2
3/3 - 2s - loss: 0.0393 - accuracy: 0.9818 - val_loss: 0.4426 - val_accuracy: 0.8957 - 2s/epoch - 565ms/step
Epoch 2/2
3/3 - 2s - loss: 0.0699 - accuracy: 0.9773 - val_loss: 0.4265 - val_accuracy: 0.8902 - 2s/epoch - 535ms/step
